<a href="https://colab.research.google.com/github/TomasMendozaHN/ICDF_Class/blob/main/05182022_FasterRCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get my Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unrar x "/content/drive/MyDrive/MaskDataset__ByTomasMendoza.rar" "/content/sample_data"

In [8]:
# Check that everything works!
import os
folder = "/content/sample_data/MaskDataset__ByTomasMendoza"
for sfolder in os.listdir(folder):
  print(sfolder)

test
new_labels
imgs


# Define some utils used by training

In [16]:
def collate_fn(batch):
    return tuple(zip(*batch))


def _flip_coco_person_keypoints(kps, width):
    flip_inds = [0, 2, 1, 4, 3, 6, 5, 8, 7, 10, 9, 12, 11, 14, 13, 16, 15]
    flipped_data = kps[:, flip_inds]
    flipped_data[..., 0] = width - flipped_data[..., 0]
    # Maintain COCO convention that if visibility == 0, then x, y = 0
    inds = flipped_data[..., 2] == 0
    flipped_data[inds] = 0
    return flipped_data


class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms
    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target


class RandomHorizontalFlip(object):
    def __init__(self, prob):
        self.prob = prob
    def __call__(self, image, target):
        if random.random() < self.prob:
            height, width = image.shape[-2:]
            image = image.flip(-1)
            bbox = target["boxes"]
            bbox[:, [0, 2]] = width - bbox[:, [2, 0]]
            target["boxes"] = bbox
            if "masks" in target:
                target["masks"] = target["masks"].flip(-1)
            if "keypoints" in target:
                keypoints = target["keypoints"]
                keypoints = _flip_coco_person_keypoints(keypoints, width)
                target["keypoints"] = keypoints
        return image, target


class ToTensor(object):
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target

# Begin importing libraries

In [ ]:
!pip install xmltodict

In [17]:
from scipy.io import loadmat
import numpy as np
import torchvision
import os
import xmltodict
from sklearn.utils import shuffle

from scipy.io import loadmat
import numpy as np
import random
import os

from torchvision.transforms import functional as F
import torch
from PIL import Image
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device("cuda:0")
print('You will train using your ', device)

You will train using your  cuda:0


In [18]:
class FaceMask_Dataset(object):
    def __init__(self, root, obj, transforms):
        self.root = root
        self.transforms = transforms

        self.img_path = os.path.join(root, 'imgs')
        self.label_path = os.path.join(root, 'new_labels')
        
        # Read labels
        self.labels = list(sorted(os.listdir(self.label_path)))
        label_names = [x[:-4] for x in self.labels]
        
        #Read Images
        self.imgs = list(sorted(os.listdir(self.img_path)))
        
        #Make both lists of images and labels match to avoid errors
        self.imgs = [x for x in self.imgs if x[:-4] in label_names]
        img_names = [x[:-4] for x in self.imgs]
        self.labels = [x for x in self.labels if x[:-4] in img_names]\
        
        if obj=='train':
            self.imgs = self.imgs[5000:]
            self.labels = self.labels[5000:]
        elif obj=='test':
            self.imgs = self.imgs[:5000]
            self.labels = self.labels[:5000]
        

    #Generates the bounding box based on the coordinates given
    def create_box_from_coordinates(self, coordinates):
        box=[int(coordinates[0][1:]),
             int(coordinates[1]),
             int(coordinates[2]),
             int(coordinates[3][:-1])]
        return [box]
    

    #Extracts the label and bounding box
    def read_label(self, path):
        #Read the xml file with the bounding boxes
        file_object  = open(path, "r")
        data = file_object.readlines()
        data = data[0].split(',')[-6:-1]
        
        label, boxes = data[0], data[1:]
        label = int(label)
        if label == 0: #if NOT wearing mask, change to label 3
            label = 3
        box = self.create_box_from_coordinates(boxes)
        return [label], box
    

    def __getitem__(self, idx):
#         print('idx: ', idx)
        
        # load images and masks
        img_path = os.path.join(self.img_path, self.imgs[idx])
        img = Image.open(img_path).convert("RGB")
        
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        label_path = os.path.join(self.label_path, self.labels[idx])
        labels, boxes = self.read_label(label_path)
        
        #Calculate number of objects in image
        n_objects=1
        iscrowd = torch.zeros((n_objects,), dtype=torch.int64)

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        image_id = torch.tensor([idx])
        
        #Calculate bounding box area
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)
        return img, target


    def __len__(self):
        return len(self.imgs)

# Define function that converts pillow images into tensors

In [19]:
#Transforms a pillow image to tensor 
def get_transform(train):
    transforms = []
    transforms.append(ToTensor())
    if train:
        transforms.append(RandomHorizontalFlip(0.5))
    return Compose(transforms)

# Faster RCNN Parameters!!!

In [22]:
# Probability threshold for your Non-Maxima Supression
box_nms_thresh = 0.5

# 0 = No Mask
# 1 = Incorrect Mask
# 2 = With Mask
num_classes = 3  

# Prepare your faster-RCNN backbone (pre-trained network)

In [ ]:
# Your must first load the previously trained FasterRCNN model from Pytorch
# first in order to fine-tune it. 
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
                                                pretrained=True, 
                                                box_nms_thresh=box_nms_thresh
                                                )

in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)   

print(model.roi_heads.box_predictor)